In [53]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os
import json
import datetime

plt.rcParams["figure.figsize"] = (20, 5)

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [54]:
timestamp = "20200826-201949"
timestamp

'20200826-201949'

In [55]:
dataset_name = "SEG_2"

In [56]:
data = np.genfromtxt("data/{}_test_set.csv".format(dataset_name), delimiter="\n", dtype=np.int64)
data

array([93617988376, 93536062752, 93747035368, ..., 92658792872,
       92658792864, 92654987192], dtype=int64)

In [57]:
dataset = np.array([data[i+1] - data[i] for i in range(int(len(data))-1)])
dataset, len(dataset)

(array([ -81925624,  210972616, -189258952, ...,   36097352,         -8,
          -3805672], dtype=int64),
 59298)

In [58]:
word_index = np.genfromtxt("data/word_index.csv", delimiter="\n", dtype=np.int64)
vocab_size = len(word_index)
vocab_size

14882

In [59]:
vocabulary = {word_index[i]:i for i in range(vocab_size)}
dict(list(vocabulary.items())[0:10])

{-1: 0,
 0: 1,
 4096: 2,
 909517620: 3,
 -909517620: 4,
 8192: 5,
 -8: 6,
 -4096: 7,
 8: 8,
 12288: 9}

In [60]:
word_index

array([       -1,         0,      4096, ...,  -7445040,  12889736,
       619958144], dtype=int64)

In [61]:
unseen_category = []
in_word_index = np.where(np.isin(dataset, word_index))[0]
for i in range(len(dataset)):
    if i in in_word_index:
        continue
    unseen_category.append(i)

In [62]:
dataset[unseen_category] = -1

In [63]:
test_set = [vocabulary[dataset[i]] for i in range(len(dataset))]
test_set = np.array(test_set).astype(np.float32)
test_set[:10]

array([ 3281., 10140., 10181.,  8177.,  9834., 14409.,  8531.,  8166.,
        6807.,  4700.], dtype=float32)

In [64]:
def generate_timeseries(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, n_feature)
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        #data.append(dataset[indices])
        labels.append(np.reshape(dataset[i:i+target_size], (target_size, 1)))
        #labels.append(dataset[i:i+target_size])
    return np.array(data), np.array(labels)

In [65]:
x_test, y_test = generate_timeseries(test_set, 0, None, 16, 8)
x_test.shape, y_test.shape

((59274, 16, 1), (59274, 8, 1))

In [66]:
x_test[0], y_test[0]

(array([[ 3281.],
        [10140.],
        [10181.],
        [ 8177.],
        [ 9834.],
        [14409.],
        [ 8531.],
        [ 8166.],
        [ 6807.],
        [ 4700.],
        [ 4453.],
        [ 3203.],
        [ 5888.],
        [11018.],
        [11563.],
        [    0.]], dtype=float32),
 array([[1.000e+00],
        [0.000e+00],
        [1.291e+04],
        [0.000e+00],
        [6.358e+03],
        [2.888e+03],
        [1.700e+02],
        [6.866e+03]], dtype=float32))

In [67]:
model = keras.models.load_model("version/{}/model.h5".format(timestamp))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_6 (Bidirection multiple                  253464    
_________________________________________________________________
dropout_2 (Dropout)          multiple                  0         
_________________________________________________________________
repeat_vector_3 (RepeatVecto multiple                  0         
_________________________________________________________________
bidirectional_7 (Bidirection multiple                  753312    
_________________________________________________________________
dropout_3 (Dropout)          multiple                  0         
_________________________________________________________________
time_distributed_3 (TimeDist multiple                  5283110   
Total params: 6,289,886
Trainable params: 6,289,886
Non-trainable params: 0
____________________________________________

In [68]:
result = model.evaluate(x_test, y_test)
result

1853/1853 [==============================] - 11s 6ms/step - loss: 11.5888 - accuracy: 0.2002


[11.58884048461914, 0.2002016007900238]

In [69]:
with open("version/{}/evaluate.txt".format(timestamp), 'w') as e:
    e.write("loss, accuracy\n")
    e.write("{}, {}".format(result[0], result[1]))